In [1]:
import os
import mne
import numpy as np
import matplotlib.pyplot as plt
import csv
import pandas as pd

from datetime import date

In [2]:
from enum import Enum
class Sensor(Enum):
    AF3 = 0
    F7 = 1
    F3 = 2
    FC5 = 3
    T7 = 4
    P7 = 5
    O1 = 6
    O2 = 7
    P8 = 8
    T8 = 9
    FC6 = 10
    F4 = 11
    F8 = 12
    AF4 = 13

In [3]:
data_dir = "data/"
out_dir  = "tf_beta/"

f0   = 13
f1   = 30
band = "BETA"

In [4]:
files = os.listdir(data_dir)

print(files)

['Morty_5_1_2018.07.23_11.33.54.edf', 'Richard_s_3_2018.07.30_16.01.47.edf', 'Alyse_2_1_2018.07.17_09.24.54.edf', 'Keaton_5_3_2018.08.08_10.59.50.edf', 'Morty_3_1_2018.07.23_11.39.49.edf', 'Keaton_1_0_2018.07.16_15.26.19.edf', 'Morty_s_3_2018.08.08_08.23.37.edf', 'Morty_5_3_2018.08.08_08.32.10.edf', 'Morty_2_3_2018.08.08_08.27.15.edf', 'Richard_5_2_2018.07.25_16.59.50.edf', 'Keaton_2_2_2018.07.26_15.24.23.edf', 'Morty_1_1_2018.07.23_11.31.04.edf', 'Keaton_3_2_2018.07.26_15.26.38.edf', 'Jay_1_2_2018.07.27_15.25.15.edf', 'Alyse_3_3_2018.08.08_10.39.08.edf', 'Alyse_4_3_2018.08.08_10.31.23.edf', 'Jay_2_1_2018.07.17_10.47.20.edf', 'Keaton_s_1_2018.07.19_10.10.22.edf', 'Alyse_3_2_2018.07.26_16.33.12.edf', 'Keaton_3_1_2018.07.19_10.21.33.edf', 'Morty_4_1_2018.07.23_11.36.57.edf', 'Morty_3_2_2018.07.30_15.19.49.edf', 'Jay_5_2_2018.07.27_15.29.46.edf', 'Richard_1_3_2018.07.30_16.31.20.edf', 'Keaton_s_2_2018.07.26_15.17.22.edf', 'Keaton_s_3_2018.08.08_10.58.06.edf', 'Keaton_3_3_2018.08.08_11.04.

In [5]:
def get_power(raw):
    """
    converts a raw MNE object that has been created using an Emotiv Pro generated EDF 
    file into a set of the average power across the specified frequencies
    
    raw: raw object
    l_freq: frequency lower bound
    h_freq: frequency upper bound
    
    return an MNE TRAverage with the average frequency from l_freq to h_freq
    """
    # define time interval over which to extract
    tmin = 0    # starting second
    tmax = 39   # ending second
    start, stop = raw.time_as_index([tmin, tmax])
    
    # define montage 
    montage = mne.channels.read_montage('standard_1020')
    raw.set_montage(montage)
    
    # add events 
    events = np.array([[0, 0, 1]]) 
    raw.add_events(events)
    
    # add picks
    picks = mne.pick_types(raw.info, eeg=True, eog=False, meg=False, stim=False, 
            exclude='bads')
    
    # add epochs
    event_id, tmin, tmax = 1, 0, 39
    baseline = (None, 0)
    epochs = mne.Epochs(raw, events, event_id, tmin, tmax,
        picks=picks,
        baseline=baseline,
        preload=True)
    
    # extract the average power across the specified range of frequencies
    freqs = np.logspace(*np.log10([f0, f1]), num=1)
    n_cycles = freqs / 2
    power, itc = mne.time_frequency.tfr_morlet(epochs, freqs=freqs, 
                                                n_cycles=n_cycles,
                                                use_fft=True,
                                                return_itc=True,
                                                decim=3, 
                                                n_jobs=1)
    
    return power

In [6]:
def preprocess_raw(raw):
    """
    prepares a MNE raw object created from an Emotiv generated EDF file to have time frequency analysis done on it
    """
    # drop unnecessary channels
    extra_ch = raw.ch_names[0:2] + raw.ch_names[16:40]
    raw.drop_channels(extra_ch)
    
    # manually add stim channel
    stim_data = np.zeros((1, len(raw.times)))
    info = mne.create_info(['STI'], raw.info['sfreq'], ['stim'])
    stim_raw = mne.io.RawArray(stim_data, info)
    raw.add_channels([stim_raw], force_update_info=True)

In [7]:
for file in files:
    raw = mne.io.read_raw_edf(data_dir + file, preload=True)
    preprocess_raw(raw)
    power = get_power(raw)
    
    d = {}
    for i in range(len(power.data)):
        d[power.ch_names[i] + "_" + band] = power.data[i][0]
    
    d["times"] = power.times
       
    df = pd.DataFrame(d)
        
    date    = date.today()
    outfile = file.split('.')[0][0:-4] + band + "_" + str(date) + ".csv"
    outpath = out_dir + outfile

    df.to_csv(outpath)

    

Extracting EDF parameters from data/Morty_5_1_2018.07.23_11.33.54.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 5119  =      0.000 ...    39.992 secs...
Creating RawArray with float64 data, n_channels=1, n_times=5120
    Range : 0 ... 5119 =      0.000 ...    39.992 secs
Ready.
1 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 projection items activated
Loading data for 1 events and 4993 original time points ...
0 bad epochs dropped
Extracting EDF parameters from data/Richard_s_3_2018.07.30_16.01.47.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 5247  =      0.000 ...    40.992 secs...
Creating RawArray with float64 data, n_channels=1, n_times=5248
    Range : 0 ... 5247 =      0.000 ...    40.992 secs
Ready.
1 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 projection items activated
Loading dat

Creating RawArray with float64 data, n_channels=1, n_times=5248
    Range : 0 ... 5247 =      0.000 ...    40.992 secs
Ready.
1 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 projection items activated
Loading data for 1 events and 4993 original time points ...
0 bad epochs dropped
Extracting EDF parameters from data/Jay_2_1_2018.07.17_10.47.20.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 5247  =      0.000 ...    40.992 secs...
Creating RawArray with float64 data, n_channels=1, n_times=5248
    Range : 0 ... 5247 =      0.000 ...    40.992 secs
Ready.
1 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 projection items activated
Loading data for 1 events and 4993 original time points ...
0 bad epochs dropped
Extracting EDF parameters from data/Keaton_s_1_2018.07.19_10.10.22.edf...
EDF file detected
Setting channel info structure...
Creating raw.info s

Not setting metadata
0 projection items activated
Loading data for 1 events and 4993 original time points ...
0 bad epochs dropped
Extracting EDF parameters from data/Richard_5_0_2018.07.16_16.49.48.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 5119  =      0.000 ...    39.992 secs...
Creating RawArray with float64 data, n_channels=1, n_times=5120
    Range : 0 ... 5119 =      0.000 ...    39.992 secs
Ready.
1 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 projection items activated
Loading data for 1 events and 4993 original time points ...
0 bad epochs dropped
Extracting EDF parameters from data/Morty_1_2_2018.07.30_15.27.35.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 5247  =      0.000 ...    40.992 secs...
Creating RawArray with float64 data, n_channels=1, n_times=5248
    Range : 0 ... 5247 =      0.000 ...    40.992 secs
Rea

EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 5247  =      0.000 ...    40.992 secs...
Creating RawArray with float64 data, n_channels=1, n_times=5248
    Range : 0 ... 5247 =      0.000 ...    40.992 secs
Ready.
1 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 projection items activated
Loading data for 1 events and 4993 original time points ...
0 bad epochs dropped
Extracting EDF parameters from data/Morty_3_3_2018.08.08_08.28.45.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 5247  =      0.000 ...    40.992 secs...
Creating RawArray with float64 data, n_channels=1, n_times=5248
    Range : 0 ... 5247 =      0.000 ...    40.992 secs
Ready.
1 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 projection items activated
Loading data for 1 events and 4993 original time points ...
0 bad epochs dropped
Extra

    Range : 0 ... 5119 =      0.000 ...    39.992 secs
Ready.
1 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 projection items activated
Loading data for 1 events and 4993 original time points ...
0 bad epochs dropped
Extracting EDF parameters from data/Alyse_s_1_2018.07.17_09.17.37.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 5247  =      0.000 ...    40.992 secs...
Creating RawArray with float64 data, n_channels=1, n_times=5248
    Range : 0 ... 5247 =      0.000 ...    40.992 secs
Ready.
1 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 projection items activated
Loading data for 1 events and 4993 original time points ...
0 bad epochs dropped
Extracting EDF parameters from data/Keaton_1_2_2018.07.26_15.29.59.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 5119  =      0.000 ...    39.992 sec

Loading data for 1 events and 4993 original time points ...
0 bad epochs dropped
Extracting EDF parameters from data/Richard_3_3_2018.07.30_16.33.51.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 5247  =      0.000 ...    40.992 secs...
Creating RawArray with float64 data, n_channels=1, n_times=5248
    Range : 0 ... 5247 =      0.000 ...    40.992 secs
Ready.
1 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 projection items activated
Loading data for 1 events and 4993 original time points ...
0 bad epochs dropped
Extracting EDF parameters from data/Alyse_4_2_2018.07.26_16.37.52.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 5247  =      0.000 ...    40.992 secs...
Creating RawArray with float64 data, n_channels=1, n_times=5248
    Range : 0 ... 5247 =      0.000 ...    40.992 secs
Ready.
1 matching events found
Applying baseline corr

Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 5119  =      0.000 ...    39.992 secs...
Creating RawArray with float64 data, n_channels=1, n_times=5120
    Range : 0 ... 5119 =      0.000 ...    39.992 secs
Ready.
1 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 projection items activated
Loading data for 1 events and 4993 original time points ...
0 bad epochs dropped
Extracting EDF parameters from data/Richard_5_3_2018.07.30_16.25.44.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 5247  =      0.000 ...    40.992 secs...
Creating RawArray with float64 data, n_channels=1, n_times=5248
    Range : 0 ... 5247 =      0.000 ...    40.992 secs
Ready.
1 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 projection items activated
Loading data for 1 events and 4993 original time points ...
0 bad epochs dropped
Extracting EDF parame